In [1]:
#imports
import requests
import pandas as pd
import os
from datetime import datetime

## config


In [2]:
# set your data folder
desti_folder = './graffiti'

# give your socrata token
token = 'YOURTOKENHERE'


In [3]:
start_date = '2019-01-01'
end_date = '2023-12-31'

In [4]:
os.makedirs(desti_folder, exist_ok=True )

In [5]:
timestamp = datetime.now()
timestamp = timestamp.strftime("%Y-%m-%d-%H-%M-%S")

In [11]:
def get_data(name, db, maxrows, parameters):
    # some key info
    db_id = db
    limit = maxrows
    i = 0
    offset_counter = 0
    pickle = True
    data_name = name
    # dict for logging
    log_dict = {}

    # base url for the request
    baseurl = ("https://data.cityofchicago.org/resource/"
            f"{db_id}.json?")

    while pickle:
        #set our params
        # note that you have to URL econde 
        params = parameters + (
            f"&$limit={limit}"
            f"&$offset={offset_counter}"
        )
        
        # make the request 
        data_req = requests.get(baseurl+params)
        #print(data_req)
        # convert to json
        data_json = data_req.json()

        # make it a df
        df = pd.json_normalize(data_json)

        # write df
        
        
        # log
        minilog = {}
        minilog['records'] = df.shape[0]
        minilog['chunk'] = i
        minilog['status'] = data_req.status_code
        minilog['offset'] = offset_counter
        log_dict[f'run_{i}'] = minilog
        log_df = pd.DataFrame.from_dict(log_dict,orient='index')
        
        if df.shape[0] == 0:
            break
        else:
            df.to_json(f'{desti_folder}/{data_name}_chunk_{i}.json',orient='records', lines=True)
            log_df.to_csv(f'{desti_folder}/{data_name}_{timestamp}_log.csv')

        # increment the chunk count
        print(i)
        i += 1

        # increment the offset
        offset_counter += limit

        # check if we need to end the loop
        if df.shape[0] == 0:
            pickle = False
        elif data_req.status_code == 200:
            pickle = True
        else: 
            pickle = False
            print(data_req.status_code)



In [22]:
graffiti_params = (
            f"$where=(created_date%20BETWEEN%20%27{start_date}T00%3A00%3A00.000%27%20AND%20%27{end_date}T23%3A59%3A59.000%27)%20AND%20sr_short_code%20not%20in%20(%27311IOC%27)"w
            f"&$order=created_date"
            )

get_data('graffiti5', 'v6vf-nfxy', 500000, parameters=graffiti_params)



<Response [200]>
0
<Response [200]>
1
<Response [200]>
2
<Response [200]>
3
<Response [200]>
4
<Response [200]>
5
<Response [200]>
6
<Response [200]>
7
<Response [200]>
8
<Response [200]>
9
<Response [200]>
10
<Response [200]>
11
<Response [200]>
